<a href="https://colab.research.google.com/github/FuyiHuang/PCA-Knowledge-Distillation-Colab/blob/main/PCA_Knowledge_Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/chiutaiyin/PCA-Knowledge-Distillation.git

Cloning into 'PCA-Knowledge-Distillation'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 124 (delta 35), reused 110 (delta 28), pack-reused 0
Receiving objects: 100% (124/124), 23.58 MiB | 16.38 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
!pip install yacs

在运行下面代码前，需要把lightweight_model.py和vgg_model.py里面的
csvfile = open('./utils/vgg_layers.csv', newline='')
改成绝对路径
content/PCA-Knowledge-Distillation/VGG backbone/utils/vgg_layers.csv

In [ ]:
import sys
sys.path.append('/content/PCA-Knowledge-Distillation/VGG backbone')
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from utils.vgg_model import VggEncoder
from utils.lightweight_model import VggSDecoder, VggSEncoder
from utils.photo_gif import GIFSmoothing
from PIL import Image
from tqdm import tqdm
import os, cv2
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
class VggEncDec(tf.keras.Model):
    def __init__(self):
        super(VggEncDec, self).__init__()
        self.sencoder = VggSEncoder()
        self.sdecoder = VggSDecoder()
        self.sencoder.load_weights('/content/PCA-Knowledge-Distillation/VGG backbone/ckpts/sencoder')
        self.sdecoder.load_weights('/content/PCA-Knowledge-Distillation/VGG backbone/ckpts/sdecoder')

    def call(self, layer, input_img):
        None

In [ ]:
def load_img(file):
    img = np.asarray(Image.open(file), dtype=np.float32)
    img = np.expand_dims(cv2.resize(img, (img.shape[1] // 8 * 8, img.shape[0] // 8 * 8)), axis=0) / 255
    return img

def inv_sqrt_cov(cov, inverse=False):
    s, u, _ = tf.linalg.svd(cov + tf.eye(cov.shape[-1]))
    n_s = tf.reduce_sum(tf.cast(tf.greater(s, 1e-5), tf.int32))
    s = tf.sqrt(s[:,:n_s])
    if inverse:
        s = 1 / s
    d = tf.linalg.diag(s)
    u = u[:,:,:n_s]
    return tf.matmul(u, tf.matmul(d, u, adjoint_b=True))

# transform: feature transformation
def stylize_core(c_feat, s_feat, transform='zca'):
    n_batch, cont_h, cont_w, n_channel = c_feat.shape
    _c_feat = tf.reshape(tf.transpose(c_feat, [0, 3, 1, 2]), [n_batch, n_channel, -1])
    if transform == 'zca':
        c_feat = stylize_zca(_c_feat, s_feat)
    elif transform == 'ot':
        c_feat = stylize_ot(_c_feat, s_feat)
    elif transform == 'adain':
        c_feat = stylize_adain(_c_feat, s_feat)

    c_feat = tf.transpose(tf.reshape(c_feat, [n_batch, n_channel, cont_h, cont_w]), [0, 2, 3, 1])
    return c_feat

def stylize_adain(c_feat, s_feat):
    m_c = tf.reduce_mean(c_feat, axis=-1, keepdims=True)
    m_s = tf.reduce_mean(s_feat, axis=-1, keepdims=True)
    c_feat = c_feat - m_c
    s_feat = s_feat - m_s
    s_c = tf.sqrt(tf.reduce_mean(c_feat * c_feat, axis=-1, keepdims=True) + 1e-8)
    s_s = tf.sqrt(tf.reduce_mean(s_feat * s_feat, axis=-1, keepdims=True) + 1e-8)
    white_c_feat = c_feat / s_c
    feat = white_c_feat * s_s + m_s
    return feat

def stylize_zca(c_feat, s_feat):
    m_c = tf.reduce_mean(c_feat, axis=-1, keepdims=True)
    m_s = tf.reduce_mean(s_feat, axis=-1, keepdims=True)
    c_feat = c_feat - m_c
    s_feat = s_feat - m_s
    c_cov = tf.matmul(c_feat, c_feat, transpose_b=True) / c_feat.shape[-1]
    s_cov = tf.matmul(s_feat, s_feat, transpose_b=True) / s_feat.shape[-1]
    inv_sqrt_c_cov = inv_sqrt_cov(c_cov, True)
    opt = tf.matmul(inv_sqrt_cov(s_cov), inv_sqrt_c_cov)
    feat = tf.matmul(opt, c_feat) + m_s
    return feat

def stylize_ot(c_feat, s_feat):
    m_c = tf.reduce_mean(c_feat, axis=-1, keepdims=True)
    m_s = tf.reduce_mean(s_feat, axis=-1, keepdims=True)
    c_feat = c_feat - m_c
    s_feat = s_feat - m_s
    c_cov = tf.matmul(c_feat, c_feat, transpose_b=True) / c_feat.shape[-1]
    s_cov = tf.matmul(s_feat, s_feat, transpose_b=True) / s_feat.shape[-1]
    sqrt_c_cov = inv_sqrt_cov(c_cov)
    inv_sqrt_c_cov = inv_sqrt_cov(c_cov, True)
    opt = inv_sqrt_cov(tf.matmul(sqrt_c_cov, tf.matmul(s_cov, sqrt_c_cov)))
    opt = tf.matmul(inv_sqrt_c_cov, tf.matmul(opt, inv_sqrt_c_cov))
    feat = tf.matmul(opt, c_feat) + m_s
    return feat

In [ ]:
# 需要把tf版本降到2.8.0
import tensorflow as tf
print(tf.__version__)
!pip uninstall tensorflow-gpu
!pip install tensorflow-gpu==2.8.0
import tensorflow as tf
print(tf.__version__)

2.8.0


In [ ]:
enc_dec = VggEncDec()

Reached target layer: relu4_1


In [ ]:
p_pro = GIFSmoothing(r=30, eps= (0.02 * 255) ** 2)

In [ ]:
!wget -O '/content/PCA-Knowledge-Distillation/VGG backbone/figures/content/0.jpg' 'https://pixabay.com/get/g6d0316fd7fbf1ec609a4650fa05f7fd6eb2633bf915552a26e1068e7f8e24a1d37d39076503183ce5ba43348f8bc0129.jpg'
!wget -O '/content/PCA-Knowledge-Distillation/VGG backbone/figures/style/0.jpg' 'https://pixabay.com/get/g6d0316fd7fbf1ec609a4650fa05f7fd6eb2633bf915552a26e1068e7f8e24a1d37d39076503183ce5ba43348f8bc0129.jpg'

--2024-08-08 07:31:47--  https://pixabay.com/get/g6d0316fd7fbf1ec609a4650fa05f7fd6eb2633bf915552a26e1068e7f8e24a1d37d39076503183ce5ba43348f8bc0129.jpg
Resolving pixabay.com (pixabay.com)... 104.18.40.96, 172.64.147.160, 2606:4700:4400::6812:2860, ...
Connecting to pixabay.com (pixabay.com)|104.18.40.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2262862 (2.2M) [image/jpeg]
Saving to: ‘/content/PCA-Knowledge-Distillation/VGG backbone/figures/content/0.jpg’

/content/PCA-Knowle 100%[===================>]   2.16M  --.-KB/s    in 0.03s   

2024-08-08 07:31:47 (77.6 MB/s) - ‘/content/PCA-Knowledge-Distillation/VGG backbone/figures/content/0.jpg’ saved [2262862/2262862]

--2024-08-08 07:31:47--  https://pixabay.com/get/g6d0316fd7fbf1ec609a4650fa05f7fd6eb2633bf915552a26e1068e7f8e24a1d37d39076503183ce5ba43348f8bc0129.jpg
Resolving pixabay.com (pixabay.com)... 104.18.40.96, 172.64.147.160, 2606:4700:4400::6812:2860, ...
Connecting to pixabay.com (pixabay.com)|104.1

In [ ]:
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
for i in tqdm(range(1)):
    cont_img = load_img(f'/content/PCA-Knowledge-Distillation/VGG backbone/figures/content/{i}.jpg')
    style_img = load_img(f'/content/PCA-Knowledge-Distillation/VGG backbone/figures/style/{i}.jpg')

    transform = 'zca'
    tf.print("image shape:", style_img.shape)
    x1 = enc_dec.sencoder(0, style_img)
    tf.print("x1 shape:", tf.shape(x1[0]))
    x2 = enc_dec.sencoder(1, x1[0])
    x3 = enc_dec.sencoder(2, x2[0])
    x4 = enc_dec.sencoder(3, x3[0])

    y1 = enc_dec.sencoder(0, cont_img)
    y2 = enc_dec.sencoder(1, y1[0])
    y3 = enc_dec.sencoder(2, y2[0])
    y4 = enc_dec.sencoder(3, y3[0])

    sfeat = tf.reshape(tf.transpose(x4[0], [0, 3, 1, 2]), [x4[0].shape[0], x4[0].shape[-1], -1])
    x = stylize_core(y4[0], sfeat, transform=transform)
    x = enc_dec.sdecoder(3, x, skip=y4[1])

    sfeat = tf.reshape(tf.transpose(x3[0], [0, 3, 1, 2]), [x3[0].shape[0], x3[0].shape[-1], -1])
    x = stylize_core(x, sfeat, transform=transform)
    x = enc_dec.sdecoder(2, x, skip=y3[1])

    sfeat = tf.reshape(tf.transpose(x2[0], [0, 3, 1, 2]), [x2[0].shape[0], x2[0].shape[-1], -1])
    x = stylize_core(x, sfeat, transform=transform)
    x = enc_dec.sdecoder(1, x, skip=y2[1])

    sfeat = tf.reshape(tf.transpose(x1[0], [0, 3, 1, 2]), [x1[0].shape[0], x1[0].shape[-1], -1])
    x = stylize_core(x, sfeat, transform=transform)
    x = tf.clip_by_value(enc_dec.sdecoder(0, x, skip=y1[1]), 0, 1)

    if not os.path.exists('/content/PCA-Knowledge-Distillation/VGG backbone/results'):
        os.mkdir('/content/PCA-Knowledge-Distillation/VGG backbone/results')
    p_pro.process(x[0], f'/content/PCA-Knowledge-Distillation/VGG backbone/figures/content/{i}.jpg').save(f'/content/PCA-Knowledge-Distillation/VGG backbone/results/{i}.jpg')

    # Display the content, style, and output images
    output_img = load_img(f'/content/PCA-Knowledge-Distillation/VGG backbone/results/{i}.jpg')

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(cont_img[0])
    ax[0].set_title('Content Image')
    ax[0].axis('off')

    ax[1].imshow(style_img[0])
    ax[1].set_title('Style Image')
    ax[1].axis('off')

    ax[2].imshow(output_img[0])
    ax[2].set_title('Output Image')
    ax[2].axis('off')

    plt.show()

  0%|          | 0/1 [00:00<?, ?it/s]

image shape: (1, 4000, 6000, 3)
x1 shape: [1 4000 6000 10]
